In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/'My Drive'/doan

/content/drive/My Drive/doan


**IMPORT DATA**

In [4]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="training",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="validation",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)


Found 21758 files belonging to 2 classes.
Using 17407 files for training.
Found 21758 files belonging to 2 classes.
Using 4351 files for validation.


In [5]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

**TRAINNING**

Model

In [6]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling,Conv2D,MaxPooling2D,BatchNormalization,LeakyReLU,GlobalAvgPool2D
from keras.models import Sequential

In [7]:
def conv_block(x, filters, kernel_size, strides, name=None):
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', name=name, use_bias=False)(x)
  x = BatchNormalization(epsilon=0.001, name=name if name is None else '%s_BatchNorm' %name)(x)
  x = LeakyReLU(alpha=0.1, name=name if name is None else '%s_LeakyRelu' %name)(x)
  return x

def residual_block(x, filters_1, filters_2, name_1, name_2):
  short_cut = x
  x = conv_block(x, filters_1, (1,1), (1,1), name=name_1)
  x = conv_block(x, filters_2, (3,3), (1,1), name=name_2)
  residual_output = short_cut + x
  return residual_output

In [8]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)

x = conv_block(inputs, filters=8, kernel_size=(3,3), strides=(1,1), name='conv1_3x3')
x = conv_block(x, filters=16, kernel_size=(3,3), strides=(2,2), name='conv2_3x3')

x = residual_block(x, filters_1=8, filters_2=16, name_1='residual1_1x1', name_2='residual1_3x3')

x = conv_block(x, filters=32, kernel_size=(3,3), strides=(2,2), name='conv4_3x3')

for i in range(2):
  x = residual_block(x, filters_1=16, filters_2=32, name_1='residual2_'+ str(i) + '_1x1', name_2='resdual2_'+ str(i) + '_3x3')

x = conv_block(x, filters=64, kernel_size=(3,3), strides=(2,2), name='conv5_3x3')

for i in range(8):
  x = residual_block(x, filters_1=32, filters_2=64, name_1='residual3_'+ str(i) + '_1x1', name_2='resdual3_'+ str(i) + '_3x3')

x = conv_block(x, filters=128, kernel_size=(3,3), strides=(2,2), name='conv6_3x3')

for i in range(8):
  x = residual_block(x, filters_1=64, filters_2=128, name_1='residual4_'+ str(i) + '_1x1', name_2='resdual4_'+ str(i) + '_3x3')

x = conv_block(x, filters=256, kernel_size=(3,3), strides=(2,2), name='conv7_3x3')

for i in range(4):
  x = residual_block(x, filters_1=128, filters_2=256, name_1='residual5_'+ str(i) + '_1x1', name_2='resdual5_'+ str(i) + '_3x3')

x = GlobalAvgPool2D()(x)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_3x3 (Conv2D)             (None, 64, 64, 8)    216         ['input_1[0][0]']                
                                                                                                  
 conv1_3x3_BatchNorm (BatchNorm  (None, 64, 64, 8)   32          ['conv1_3x3[0][0]']              
 alization)                                                                                       
                                                                                                  
 conv1_3x3_LeakyRelu (LeakyReLU  (None, 64, 64, 8)   0           ['conv1_3x3_BatchNorm[0][0]']

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard, LearningRateScheduler

In [10]:
loss = tf.keras.losses.BinaryCrossentropy()
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

In [11]:
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])

# Fit model (training)
history = model.fit(train_ds, steps_per_epoch=len(train_ds),
    validation_data=val_ds, validation_steps=len(val_ds), epochs=30, verbose=1,callbacks=callbacks)

Epoch 1/40
544/544 [==============================] - 1629s 3s/step - loss: 0.2363 - accuracy: 0.9076 - val_loss: 0.1302 - val_accuracy: 0.9531 - lr: 0.0010
Epoch 2/40
544/544 [==============================] - 52s 96ms/step - loss: 0.0851 - accuracy: 0.9691 - val_loss: 0.3273 - val_accuracy: 0.8878 - lr: 0.0010
Epoch 3/40
544/544 [==============================] - 51s 93ms/step - loss: 0.0466 - accuracy: 0.9835 - val_loss: 0.0366 - val_accuracy: 0.9890 - lr: 0.0010
Epoch 4/40
544/544 [==============================] - 51s 93ms/step - loss: 0.0395 - accuracy: 0.9870 - val_loss: 0.1698 - val_accuracy: 0.9529 - lr: 0.0010
Epoch 5/40
544/544 [==============================] - 51s 93ms/step - loss: 0.0311 - accuracy: 0.9901 - val_loss: 0.0431 - val_accuracy: 0.9855 - lr: 0.0010
Epoch 6/40
544/544 [==============================] - 51s 93ms/step - loss: 0.0287 - accuracy: 0.9908 - val_loss: 0.0456 - val_accuracy: 0.9848 - lr: 0.0010
Epoch 7/40
544/544 [==============================] - 51s 